# Evaluate a MedCATtrainer project export

In [1]:
import json
import pandas as pd
import plotly.graph_objects as go
from plotly. subplots import make_subplots
from IPython.display import Image
from collections import Counter

In [ ]:
mct_export = '../../data/medcattrainer_export/'+''  # mct_export .json here
with open(mct_export, 'r') as jsonfile:
    mct = json.load(jsonfile)

In [ ]:
# projects
for p in mct['projects']:
    print(p['name'])

In [ ]:
# documents
doc_lst = []
for p in mct['projects']:
    for doc in p['documents']:
        doc_lst.append(doc['name'])
doc_lst

In [2]:
# annotations
ann_lst = []
for p in mct['projects']:
    for doc in p['documents']:
        for anns in doc['annotations']:
            ann_lst.append(anns)
ann_lst


NameError: name 'mct' is not defined

# Summary format for analysis

In [ ]:

ann_lst = []
for p in mct['projects']:
    for doc in p['documents']:
        for anns in doc['annotations']:
            output = dict()
            output['project'] = p['name']
            output['document_name'] = doc['name']
            meta_anns_dict = {}
            for meta_ann in anns['meta_anns'].items():
                meta_anns_dict.update({meta_ann[0]: meta_ann[1]['value']})
                
            anns.pop('meta_anns')
            output.update(anns)
            output.update(meta_anns_dict)
            ann_lst.append(output)
final_output = pd.DataFrame(ann_lst)
final_output['last_modified'] = pd.to_datetime(final_output['last_modified'])

In [ ]:
final_output

In [ ]:
# Counts of annotations
cui_counts = Counter(final_output['cui'])
cui_counts

# Make User plots

In [ ]:
df = final_output[['user', 'last_modified']]
data = df.groupby([df['last_modified'].dt.year.rename('year'),
                   df['last_modified'].df.month.rename('month'),
                   df['last_modified'].dt.day.rename('day'),
                   df['user']]).agg({'count'})

data = pd.DataFrame(data)
data.columns = data.columns.droplevel()
data = data.reset_index(drop=False)
data['date'] = pd.datetime(data[['year', 'month', 'day']])

In [ ]:
data

## Plot

In [ ]:
# annotator work

fig = go.Figure()
for user in data['user'].unique():
    fig.add_trace(
        go.Bar(x=data[data['user'] == user]['date'], y=data[data['user'] == user]['count'], name=user),
    )

fig.update_layout(tutle = {'text': 'MedCATtrainer Annotator Progress'})
fig.update_layout(legend_title_text='MedCAT Annotator')
fig.update_xaxes(title_text='Date')
fig.update_yaxes(title_text='Annotation Count')
fig.show()